# Avsc Builder
The avsc oprojet is used to dynamically generate avro schemas based on certain information. Normally we can say get csv contianing tables and based on that gfo build a schema.


In [ ]:
# import everything
from logging import Logger, FileHandler
from pathlib import Path
from re import compile, escape
from typing import Union, List
from pandas import read_csv

In [ ]:
# Create a logger
logger = Logger("me")

In [ ]:
# Create file finder

def find_files(
    root_dir:Union[str,Path],
    regex_file_name_pattern:str,
    file_format:str="csv"
    ) -> List[Path]:
  """
    Using a specific directory traverse the directory
    and find all files ending with a specifc file format
    Each of those found files need to be valid regarding some pattern

    Args:


    Raises:


    Returns:
      List of file paths

  """
  if isinstance(root_dir,str):
    root_dir = Path(root_dir)
  if not root_dir.exists():
    err_msg = f"The root directory {str(root_dir)} does not exist"
    logger.error(err_msg)
    raise FileNotFoundError(err_msg)
  if not root_dir.is_dir():
    err_msg = f"{root_dir} is not a directoy"
    logger.error(err_msg)
    raise NotADirectoryError(err_msg)

  pattern = compile(rf"{regex_file_name_pattern}.*\.{file_format}$")
  found_files:List[Path] = []
  for found_file in root_dir.rglob(f"*.{file_format}"):
    if pattern.search(found_file.name):
      found_files.append(found_file)

  return found_files



In [ ]:
db2_to_avro = {
    'decimal': 'decimal',
    'numeric': 'decimal',
    'char': 'string',
    'int': 'int',
    'boolean': 'boolean',
    'date': 'date',
    'timestamp': 'timestamp'
}

def map_to_avro(df, db_type):
    factory = {"db2": db2_to_avro}
    df['avrotype'] = df['datatype'].map(db2_to_avro)
    return df


In [ ]:

def build(df):
  df = map_to_avro(df=df,db_type="db2")
  df["avrotype"].fillna("string", inplace=True)
  avro_fields = []
  for index, row in df.iterrows():
      avro_data_type = row['avrotype']

      if avro_data_type == 'decimal':
          avro_type = {
              'type': 'bytes',
              'logicalType': 'decimal',
              'precision': 38,
              'scale': 18
          }
      elif avro_data_type == 'int':
          avro_type = {
              'type': 'bytes',
              'logicalType': 'decimal',
              'precision': 38,
              'scale': 0
          }
      else:
          avro_type = 'string'

      avro_field = {
          'name': row['fieldname'],
          'type': [avro_type, 'null']
      }
      avro_fields.append(avro_field)
  avro_schema = {
        " type " : "record",
        " namespace " : "Tutorialspoint",
        " name " : "myBanantest", # HARDCODE HERE
        " fields " : avro_fields
      }
  return avro_schema


In [ ]:
def write_back(avro_schema:str,file_path:Path, file_name:str, file_format:str):
  pass

def lower_columns(df):
  for column in df.columns:
    df[column] = df[column].apply(lambda val: val.lower() if isinstance(val,str) else val)
  return df

for i in find_files(root_dir="./top",file_format="csv",regex_file_name_pattern="test"):
  df = read_csv(i)
  df.columns = df.columns.str.lower()

  df = lower_columns(df)
  avsc = build(df)
  print(avsc)

[{'name': 'aaa', 'type': [{'type': 'bytes', 'logicalType': 'decimal', 'precision': 38, 'scale': 18}, 'null']}, {'name': 'bbb', 'type': [{'type': 'bytes', 'logicalType': 'decimal', 'precision': 38, 'scale': 18}, 'null']}, {'name': 'cccccccccccccccccccccccccccc', 'type': [{'type': 'bytes', 'logicalType': 'decimal', 'precision': 38, 'scale': 18}, 'null']}, {'name': 'dddddddddddddddddddddddddddddd', 'type': ['string', 'null']}, {'name': 'julian', 'type': ['string', 'null']}, {'name': 'srikar', 'type': ['string', 'null']}, {'name': 'other', 'type': ['string', 'null']}, {'name': 'eating', 'type': ['string', 'null']}, {'name': 'dancing', 'type': ['string', 'null']}]
